In [6]:
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt
import random
from tqdm import tqdm

sequences = []
generated_sequences = set()


def complement_base(base):
       if base == 'A':
              return 'T'
       elif base == 'T':
              return 'A'
       elif base == 'C':
              return 'G'
       else:
              return 'C'

def reverse_compliment(seq):
    seq_dict = {'A':'T', 'T':'A','C':'G', 'G':'C'}
    reverse_compliment = ''
    for base in seq[::-1]:
        if base == ' ': #if block to clean up the spacing in the data above, if empty space then add an empty space
            reverse_compliment += ' '
        else: #if not empty space look up the dictionary
            reverse_compliment += seq_dict[base]
    return reverse_compliment
    
for i in tqdm(range(5000000)):
       cg_count = 0
       sequence = ''
       reverse_compl = ''
       prev_base = ""
       base_repeated = 0
       for j in range(8):
              if j == 0:
                     base = random.choice(['A','G','C','T'])
              elif j != 0:
                     base = random.choice(['A','T','C','G'])
                     while base == prev_base and base_repeated == 1:
                            base = random.choice(['A','T','C','G']) 
              
              if base == prev_base:
                     base_repeated += 1
              else:
                     base_repeated = 0
              
              if base in ['C', 'G']:
                     cg_count += 1
              if cg_count == 5:
                     break
              
              prev_base = base
              sequence += base
              reverse_compl = complement_base(base) + reverse_compl
              
       if sequence not in generated_sequences and sequence != reverse_compl:
              sequences.append(sequence)
              generated_sequences.add(sequence)
              generated_sequences.add(reverse_compl)

#      '''DEFAULT PARAMETERS''' seq, check=True, strict=True, c_seq=None, shift=0, nn_table=None, tmm_table=None, 
# imm_table=None, de_table=None, dnac1=25, dnac2=25, selfcomp=False, Na=50, K=0, Tris=0, Mg=0, dNTPs=0, saltcorr=5
forwards = []

for sequence in sequences:
       Tm = mt.Tm_NN(sequence, Mg = 10, Na = 5) 
       if 15.4 <= Tm <= 16.99:
              forwards.append(sequence)

print(len(forwards))

100%|█████████████████████████████| 5000000/5000000 [00:36<00:00, 135351.54it/s]


6219


In [ ]:
import random
from nupack import *


model1 = Model(material='dna', celsius=16, sodium=0.05, magnesium=0.01)
max_unique_seq = []
max_len = 0

r_list = []
while len(r_list) < len(forwards):
    unique_seq = []
    potential_seq = []

    r = random.randint(0,len(forwards)-1)
    if r not in r_list:
        r_list.append(r)
        if len(unique_seq) == 0:
            unique_seq.append(forwards[r])
            unique_seq.append(reverse_complement(forwards[r]))

        
        for a in tqdm(range(1000000)):
            r1_list = []
            if len(r1_list) == len(forwards)-1:
                continue
            count = 0
            for b, uni_seq in enumerate(unique_seq):
                if b % 2 ==0:
                    
                    r1 = random.randint(0,len(forwards)-1)
                    if r1 not in r1_list and r1 != r:
                        potential_seq.append(forwards[r1])
                        r1_list.append(r1)

                        dG_comp_pot = pfunc((forwards[r1], reverse_complement(forwards[r1])), model=model1)
                        
                        dG_pot_f_1 = pfunc((forwards[r1], uni_seq), model = model1) #compare dG from possible pair (forward) to the sequences inside unique_seq
                        
                        dG_pot_f_2 = pfunc((forwards[r1], reverse_complement(uni_seq)), model = model1)
                        
                        dG_pot_r_1 = pfunc((reverse_complement(forwards[r1]),uni_seq),model=model1) #compare dG from possible pair (reverse) to the sequences inside unique_seq
                        
                        dG_pot_r_2 = pfunc((reverse_complement(forwards[r1]),reverse_complement(uni_seq)), model=model1)

                        ddG_pot_f_1 = dG_comp_pot[1] - dG_pot_f_1[1]
                        ddG_pot_f_2 = dG_comp_pot[1] - dG_pot_f_2[1]

                        ddG_pot_r_1 = dG_comp_pot[1] - dG_pot_r_1[1]
                        ddG_pot_r_2 = dG_comp_pot[1] - dG_pot_r_2[1]
                        
                        if ddG_pot_f_1 <= -5 and ddG_pot_f_2 <= -5 and ddG_pot_r_1 <= -5 and ddG_pot_r_2 <= -5:
                            # print(ddG_pot_f_1,',',ddG_pot_f_2,',',ddG_pot_r_1,',',ddG_pot_r_2)
                            count += 2
                        # print(count)
                        if count == len(unique_seq): #why unique_seq cannot more than 4
                            # print(len(unique_seq))
                            
                            count_1 = 0
                            for c, strand in enumerate(unique_seq):
                                
                                if c % 2 ==0:
                                    dG_comp_1 = pfunc((strand, reverse_complement(strand)), model=model1) #compare dG from the sequences inside unique_seq to the possible pair
                                    
                                    dG_forw_1 = pfunc((strand,potential_seq[0]), model=model1) #check forward strand inside unique with forward and reverse in the possible pair
                                    dG_forw_2 = pfunc((strand,reverse_complement(potential_seq[0])),model=model1)
                                    

                                    dG_rev_1 = pfunc((reverse_complement(strand),potential_seq[0]), model=model1) #check reverse strand inside unique with forward and reverse in the possible pair
                                    dG_rev_2 = pfunc((reverse_complement(strand),reverse_complement(forwards[r1])),model=model1)
                                    

                                    ddG_forw_1 = dG_comp_1[1] - dG_forw_1[1]
                                    ddG_forw_2 = dG_comp_1[1] - dG_forw_2[1]
                                    

                                    ddG_rev_1 = dG_comp_1[1] - dG_rev_1[1]
                                    ddG_rev_2 = dG_comp_1[1] - dG_rev_2[1]
                                    

                                    if ddG_forw_1 <= -5 and ddG_forw_2 <= -5 and ddG_rev_1 <= -5 and ddG_rev_2 <= -5:
                                        # print(ddG_forw_1,',',ddG_forw_2,',',ddG_rev_1,',',ddG_rev_2)
                                        count_1 += 2   #why always = 2
                                    
                                    if count_1 == len(unique_seq):
                                        
                                        unique_seq.append(potential_seq[0])
                                        unique_seq.append(reverse_complement(potential_seq[0]))
                                        

                            if c == len(unique_seq)-1:
                                potential_seq = []
                            
            with open('forwards_list', 'w') as file:
                for number in r_list:
                    file.write(str(number) + ',')
                
                
            if len(unique_seq) > max_len:
                max_len = len(unique_seq)
                max_unique_seq = unique_seq[:]
                with open('24_unique_strands.txt', 'w') as file:
                    for i in range(0, len(max_unique_seq), 2):
                        print(max_unique_seq[i])
                        print(max_unique_seq[i+1])
                        file.write(max_unique_seq[i] + '\n')
                        file.write(max_unique_seq[i+1] + '\n')
                    
                






  0%|                                    | 2/1000000 [00:00<19:41:40, 14.10it/s]

CATCTACTC
GAGTAGATG
ATCTCATGA
TCATGAGAT
TCTAAGTGA
TCACTTAGA
GAAGCTATT
AATAGCTTC
ACATAGGAA
TTCCTATGT
CATCTACTC
GAGTAGATG
ATCTCATGA
TCATGAGAT
TCTAAGTGA
TCACTTAGA
GAAGCTATT
AATAGCTTC
ACATAGGAA
TTCCTATGT
AGCAATAAG
CTTATTGCT


  0%|                                    | 6/1000000 [00:00<22:42:13, 12.23it/s]

CATCTACTC
GAGTAGATG
ATCTCATGA
TCATGAGAT
TCTAAGTGA
TCACTTAGA
GAAGCTATT
AATAGCTTC
ACATAGGAA
TTCCTATGT
AGCAATAAG
CTTATTGCT
GCTGTCAT
ATGACAGC
CATCTACTC
GAGTAGATG
ATCTCATGA
TCATGAGAT
TCTAAGTGA
TCACTTAGA
GAAGCTATT
AATAGCTTC
ACATAGGAA
TTCCTATGT
AGCAATAAG
CTTATTGCT
GCTGTCAT
ATGACAGC
AATTACACC
GGTGTAATT


  0%|                                   | 11/1000000 [00:00<21:07:04, 13.15it/s]

...

  0%|                                 | 2630/1000000 [05:32<66:45:18,  4.15it/s]

CATCTACTC
GAGTAGATG
ATCTCATGA
TCATGAGAT
TCTAAGTGA
TCACTTAGA
GAAGCTATT
AATAGCTTC
ACATAGGAA
TTCCTATGT
AGCAATAAG
CTTATTGCT
GCTGTCAT
ATGACAGC
AATTACACC
GGTGTAATT
GTCGATAAT
ATTATCGAC
TAATAACGG
CCGTTATTA
GATTCCGA
TCGGAATC
GTACTAGTC
GACTAGTAC
ACCACTAAT
ATTAGTGGT
ATACAACAG
CTGTTGTAT
CATGTTACT
AGTAACATG
TCGTCTCT
AGAGACGA
TATTCTTGC
GCAAGAATA
CCAGATATG
CATATCTGG
ACCATACAT
ATGTATGGT
TGGTCCTT
AAGGACCA
GTTCTAACA
TGTTAGAAC
GGCAGTTA
TAACTGCC
CTTGTTCG
CGAACAAG
ACCTTCATA
TATGAAGGT
GCATTGTC
GACAATGC
TCGTATAGT
ACTATACGA


  1%|▏                                | 5603/1000000 [12:39<56:11:26,  4.92it/s]

CATCTACTC
GAGTAGATG
ATCTCATGA
TCATGAGAT
TCTAAGTGA
TCACTTAGA
GAAGCTATT
AATAGCTTC
ACATAGGAA
TTCCTATGT
AGCAATAAG
CTTATTGCT
GCTGTCAT
ATGACAGC
AATTACACC
GGTGTAATT
GTCGATAAT
ATTATCGAC
TAATAACGG
CCGTTATTA
GATTCCGA
TCGGAATC
GTACTAGTC
GACTAGTAC
ACCACTAAT
ATTAGTGGT
ATACAACAG
CTGTTGTAT
CATGTTACT
AGTAACATG
TCGTCTCT
AGAGACGA
TATTCTTGC
GCAAGAATA
CCAGATATG
CATATCTGG
ACCATACAT
ATGTATGGT
TGGTCCTT
AAGGACCA
GTTCTAACA
TGTTAGAAC
GGCAGTTA
TAACTGCC
CTTGTTCG
CGAACAAG
ACCTTCATA
TATGAAGGT
GCATTGTC
GACAATGC
TCGTATAGT
ACTATACGA
CGCCTTAT
ATAAGGCG


  1%|▏                                | 5883/1000000 [13:19<40:45:27,  6.78it/s]